###  Given a query from an online healthcare platform, identify the intent class the query belongs to.
- Using Different pretrained model
- for multiple language by directly using the source language as well converting them to language

## Intent Recognition in English Queries Using BioClinicalBERT as pre trained model 
- On Two Dataset of indic-health-demo 
    - ihiqid-1mg
    - ihqid-webmd

In [1]:
# Read the Dataset
import pandas as pd
df=pd.read_csv('/kaggle/input/ihqid-1mg/train.csv')
df3=pd.read_csv('/kaggle/input/ihqid-1mg1/test.csv')
y_train=df['Manual_Intent']
X_train=df['question_english']
y_test=df3['Manual_Intent']
X_test=df3['question_english']

In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
#print(y_train)
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=40,
    output_dir="/kaggle/working",
    num_train_epochs=6,
    load_best_model_at_end=True,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy
40,No log,0.739988,0.687500
80,No log,0.683072,0.758929
120,No log,0.755159,0.803571
160,No log,0.860728,0.812500
200,No log,0.920706,0.794643


{'eval_loss': 0.9402279257774353, 'eval_accuracy': 0.8125, 'eval_runtime': 9.8308, 'eval_samples_per_second': 11.393, 'eval_steps_per_second': 1.424, 'epoch': 6.0}
              precision    recall  f1-score   support

     disease       0.71      0.83      0.77        24
        drug       0.91      0.96      0.94        54
       other       0.50      0.15      0.24        13
   treatment       0.74      0.81      0.77        21

    accuracy                           0.81       112
   macro avg       0.72      0.69      0.68       112
weighted avg       0.79      0.81      0.79       112



### Intent Reconginition on ihqid_webmd Dataset

In [3]:
webmd_train=pd.read_csv('/kaggle/input/ihqid-webmd/train.csv')
webmd_test=pd.read_csv('/kaggle/input/ihqid-webmd/test.csv')
# extract only reuired english columns
#df1=webmd_train[['question_english','disease_english','drug_english','treatment_english']]
#df2=webmd_test[['question_english','disease_english','drug_english','treatment_english']]
#df1['intent'] = df1[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)
#df2['intent'] = df2[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)
y_train=webmd_train['Manual_Intent']
X_train=webmd_train['question_english']
y_test=webmd_test['Manual_Intent']
X_test=webmd_test['question_english']

In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
print(y_train)
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=40,
    output_dir="/kaggle/working",
    num_train_epochs=3,
    load_best_model_at_end=True,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0                drug
1               other
2                drug
3             disease
4             disease
            ...      
715             other
716    treatment plan
717    treatment plan
718    treatment plan
719    treatment plan
Name: Manual_Intent, Length: 720, dtype: object


Step,Training Loss,Validation Loss,Accuracy
40,No log,1.183671,0.560166
80,No log,0.818702,0.721992
120,No log,0.769937,0.717842
160,No log,0.784351,0.738589
200,No log,0.746537,0.751037
240,No log,0.746125,0.763485


{'eval_loss': 0.7441739439964294, 'eval_accuracy': 0.7717842323651453, 'eval_runtime': 21.5213, 'eval_samples_per_second': 11.198, 'eval_steps_per_second': 1.44, 'epoch': 3.0}
                precision    recall  f1-score   support

       disease       0.78      0.84      0.81        76
          drug       0.75      0.81      0.78        53
         other       0.79      0.77      0.78        73
treatment plan       0.74      0.59      0.66        39

      accuracy                           0.77       241
     macro avg       0.77      0.75      0.76       241
  weighted avg       0.77      0.77      0.77       241



## Intent Recognition in English Queries Using RoBERTa as pre trained model 

In [5]:
import pandas as pd
df=pd.read_csv('/kaggle/input/ihqid-1mg/train.csv')
df3=pd.read_csv('/kaggle/input/ihqid-1mg1/test.csv')
y_train=df['Manual_Intent']
X_train=df['question_english']
y_test=df3['Manual_Intent']
X_test=df3['question_english']

In [6]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "roberta-base"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=60,
    output_dir="/kaggle/working",
    num_train_epochs=3,
    load_best_model_at_end=True,
    save_steps=600,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
60,No log,0.884299,0.705357


{'eval_loss': 0.7832251191139221, 'eval_accuracy': 0.7589285714285714, 'eval_runtime': 10.0008, 'eval_samples_per_second': 11.199, 'eval_steps_per_second': 1.4, 'epoch': 3.0}
              precision    recall  f1-score   support

     disease       0.59      0.92      0.72        24
        drug       0.89      1.00      0.94        54
       other       0.33      0.15      0.21        13
   treatment       0.88      0.33      0.48        21

    accuracy                           0.76       112
   macro avg       0.67      0.60      0.59       112
weighted avg       0.76      0.76      0.72       112



### Intent Reconginition on ihqid_webmd Dataset 

In [7]:
webmd_train=pd.read_csv('/kaggle/input/ihqid-webmd/train.csv')
webmd_test=pd.read_csv('/kaggle/input/ihqid-webmd/test.csv')
y_train=webmd_train['Manual_Intent']
X_train=webmd_train['question_english']
y_test=webmd_test['Manual_Intent']
X_test=webmd_test['question_english']

In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "roberta-base"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=90,
    output_dir="/kaggle/working",
    num_train_epochs=2,
    load_best_model_at_end=True,
    save_steps=900,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
90,No log,0.808182,0.742739
180,No log,0.721608,0.771784


{'eval_loss': 0.72160804271698, 'eval_accuracy': 0.7717842323651453, 'eval_runtime': 21.8293, 'eval_samples_per_second': 11.04, 'eval_steps_per_second': 1.42, 'epoch': 2.0}
                precision    recall  f1-score   support

       disease       0.83      0.71      0.77        76
          drug       0.78      0.89      0.83        53
         other       0.75      0.77      0.76        73
treatment plan       0.71      0.74      0.72        39

      accuracy                           0.77       241
     macro avg       0.77      0.78      0.77       241
  weighted avg       0.77      0.77      0.77       241



## Intent Recognition in English Queries Using SVM

In [9]:
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
df=pd.read_csv('/kaggle/input/ihqid-1mg/train.csv')
df3=pd.read_csv('/kaggle/input/ihqid-1mg1/test.csv')
y_train=df['Manual_Intent']
X_train=df['question_english']
y_test=df3['Manual_Intent']
X_test=df3['question_english']

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Train an SVM model for the intent
svm_model = SVC()
svm_model.fit(X_tfidf, y_train)

# Transform X_test using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Predict the "drug" intent for the test set
y_pred_drug_intent = svm_model.predict(X_test_tfidf)

# Print the classification report
report = classification_report(y_test, y_pred_drug_intent, target_names=['drug', 'disease','treatment','other'])
print(report)

              precision    recall  f1-score   support

        drug       0.47      0.83      0.60        24
     disease       0.81      1.00      0.89        54
   treatment       0.50      0.08      0.13        13
       other       0.00      0.00      0.00        21

    accuracy                           0.67       112
   macro avg       0.44      0.48      0.41       112
weighted avg       0.55      0.67      0.57       112



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Intent Reconginition on ihqid_webmd Dataset using svm

In [11]:
webmd_train=pd.read_csv('/kaggle/input/ihqid-webmd/train.csv')
webmd_test=pd.read_csv('/kaggle/input/ihqid-webmd/test.csv')
# extract only reuired english columns
#df1=webmd_train[['question_english','disease_english','drug_english','treatment_english']]
#df2=webmd_test[['question_english','disease_english','drug_english','treatment_english']]
#df1['intent'] = df1[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)
#df2['intent'] = df2[['disease_english','drug_english','treatment_english']].notna().idxmax(axis=1)
y_train=webmd_train['Manual_Intent']
X_train=webmd_train['question_english']
y_test=webmd_test['Manual_Intent']
X_test=webmd_test['question_english']

In [12]:
#train and report on the ihiqid_webmd data
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Train an SVM model for the "drug" intent
svm_model = SVC()
svm_model.fit(X_tfidf, y_train)

# Transform X_test using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Predict the "drug" intent for the test set
y_pred_drug_intent = svm_model.predict(X_test_tfidf)

# Print the classification report
report = classification_report(y_test, y_pred_drug_intent, target_names=['drug', 'disease','treatment','other'])
print(report)

              precision    recall  f1-score   support

        drug       0.59      0.82      0.69        76
     disease       0.62      0.57      0.59        53
   treatment       0.64      0.64      0.64        73
       other       0.86      0.31      0.45        39

    accuracy                           0.63       241
   macro avg       0.68      0.58      0.59       241
weighted avg       0.65      0.63      0.61       241



## Intent Recognition in Indic language Queries Using RoBERTa as pre trained model by translating it to English

In [22]:
! pip install deep-translator

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for deep-translator from https://files.pythonhosted.org/packages/38/3f/61a8ef73236dbea83a1a063a8af2f8e1e41a0df64f122233938391d0f175/deep_translator-1.11.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00


In [25]:
import pandas as pd
# data processing ihqid-1mg
df=pd.read_csv('/kaggle/input/ihqid-1mg/train.csv')
df3=pd.read_csv('/kaggle/input/ihqid-1mg1/test.csv')
# extract only the needed columns
#df1=df[['question_hindi','disease_hindi','drug_hindi','treatment_hindi']]
#df2=df3[['question_hindi','disease_hindi','drug_hindi','treatment_hindi']]
# drop the datapoints having the null in all of its intent is null
#df1=df1.dropna(subset=['disease_hindi','drug_hindi','treatment_hindi'],how='all')
#df2=df2.dropna(subset=['disease_hindi','drug_hindi','treatment_hindi'],how='all')
# create a column name intent to label the trainig point
#df1['Manual_Intent'] = df1[['disease_hindi','drug_hindi','treatment_hindi']].notna().idxmax(axis=1)
#df2['intent'] = df2[['disease_hindi','drug_hindi','treatment_hindi']].notna().idxmax(axis=1)
# find the feature and lable as train and test data
y_train=df['Manual_Intent']
X_train=df['question_hindi']
y_test=df3['Manual_Intent']
X_test=df3['question_hindi']
from deep_translator import GoogleTranslator

google_translator = GoogleTranslator(source='hi', target='en')

def translate_to_english(text):
    translated = google_translator.translate(text)
    return translated

In [26]:
# perform back translation
y_train=list(map(translate_to_english,y_train))
X_train=list(map(translate_to_english,X_train))
y_test=list(map(translate_to_english,y_test))
X_test=list(map(translate_to_english,X_test))

In [27]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
#print(y_train)
train_dataset = IntentDataset(X_train, y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test, y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=40,
    output_dir="/kaggle/working",
    num_train_epochs=3,
    load_best_model_at_end=True,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
40,No log,0.860281,0.651786
80,No log,0.705542,0.687500


{'eval_loss': 0.6349409818649292, 'eval_accuracy': 0.7946428571428571, 'eval_runtime': 10.5842, 'eval_samples_per_second': 10.582, 'eval_steps_per_second': 1.323, 'epoch': 3.0}
              precision    recall  f1-score   support

     disease       0.59      0.96      0.73        24
       drugs       0.92      1.00      0.96        54
       other       0.00      0.00      0.00        13
   treatment       0.86      0.57      0.69        21

    accuracy                           0.79       112
   macro avg       0.59      0.63      0.59       112
weighted avg       0.73      0.79      0.75       112



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Intent Reconginition on ihqid_webmd Dataset by translating in to english

In [28]:
# data processing ihqid-webmd
df=pd.read_csv('/kaggle/input/ihqid-webmd/train.csv')
df3=pd.read_csv('/kaggle/input/ihqid-webmd/test.csv')
y_train=df['Manual_Intent']
X_train=df['question_hindi']
y_test=df3['Manual_Intent']
X_test=df3['question_hindi']

#translate 
# perform back translation
y_train=list(map(translate_to_english,y_train))
X_train=list(map(translate_to_english,X_train))
y_test=list(map(translate_to_english,y_test))
X_test=list(map(translate_to_english,X_test))

In [29]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
#print(y_train)
train_dataset = IntentDataset(X_train, y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test, y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=40,
    output_dir="/kaggle/working",
    num_train_epochs=2,
    load_best_model_at_end=True,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
40,No log,1.048576,0.663900
80,No log,0.767935,0.755187
120,No log,0.780963,0.713693
160,No log,0.707363,0.767635


{'eval_loss': 0.6869829297065735, 'eval_accuracy': 0.7759336099585062, 'eval_runtime': 22.103, 'eval_samples_per_second': 10.903, 'eval_steps_per_second': 1.403, 'epoch': 2.0}
                precision    recall  f1-score   support

       disease       0.84      0.83      0.83        76
         drugs       0.76      0.79      0.78        53
         other       0.78      0.78      0.78        73
treatment plan       0.66      0.64      0.65        39

      accuracy                           0.78       241
     macro avg       0.76      0.76      0.76       241
  weighted avg       0.78      0.78      0.78       241



## Intent Recognition in Indic language Queries Using xlm-roberta-base as pre trained model

In [30]:
import pandas as pd
# data processing ihqid-1mg
df=pd.read_csv('/kaggle/input/ihqid-1mg/train.csv')
df3=pd.read_csv('/kaggle/input/ihqid-1mg1/test.csv')
y_train=df['Manual_Intent']
X_train=df['question_hindi']
y_test=df3['Manual_Intent']
X_test=df3['question_hindi']

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "xlm-roberta-base"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
print(y_train)
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=37,
    output_dir="/kaggle/working",
    num_train_epochs=8,
    load_best_model_at_end=True,
    save_steps=740,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0           drug
1           drug
2          other
3           drug
4        disease
         ...    
300         drug
301    treatment
302        other
303        other
304    treatment
Name: Manual_Intent, Length: 305, dtype: object


Step,Training Loss,Validation Loss,Accuracy
37,No log,1.115316,0.482143
74,No log,1.018675,0.616071
111,No log,0.976539,0.616071
148,No log,0.879504,0.642857
185,No log,0.892882,0.669643
222,No log,0.798559,0.696429
259,No log,1.068658,0.642857
296,No log,0.821479,0.696429


{'eval_loss': 0.8134545683860779, 'eval_accuracy': 0.6964285714285714, 'eval_runtime': 10.6515, 'eval_samples_per_second': 10.515, 'eval_steps_per_second': 1.314, 'epoch': 8.0}
              precision    recall  f1-score   support

     disease       0.46      0.92      0.61        24
        drug       0.88      0.98      0.93        54
       other       0.00      0.00      0.00        13
   treatment       0.75      0.14      0.24        21

    accuracy                           0.70       112
   macro avg       0.52      0.51      0.45       112
weighted avg       0.66      0.70      0.62       112



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Intent Reconginition on ihqid_webmd Dataset on indic language multilingual pre-trained model

In [34]:
import pandas as pd
# data processing ihqid-webmd
df1=pd.read_csv('/kaggle/input/ihqid-webmd/train.csv')
df2=pd.read_csv('/kaggle/input/ihqid-webmd/test.csv')
y_train=df1['Manual_Intent']
X_train=df1['question_hindi']
y_test=df2['Manual_Intent']
X_test=df2['question_hindi']

In [35]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments


# Encode the intent labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_t_encoded = label_encoder.fit_transform(y_test)

# Load the RoBERTa tokenizer and model
model_name = "xlm-roberta-base"  # You can choose a specific RoBERTa model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize and encode the training and testing data
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
print(y_train)
train_dataset = IntentDataset(X_train.tolist(), y_encoded, tokenizer, max_length=64)
test_dataset = IntentDataset(X_test.tolist(), y_t_encoded, tokenizer, max_length=64)

# Define training arguments and a trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=90,
    output_dir="/kaggle/working",
    num_train_epochs=8,
    load_best_model_at_end=True,
    save_steps=900,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()},
)

# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Generate predictions for the test set
y_pred_intent = trainer.predict(test_dataset).predictions.argmax(axis=1)

# Decode the encoded labels
y_pred_intent_decoded = label_encoder.inverse_transform(y_pred_intent)
# Print the classification report
report = classification_report(y_test, y_pred_intent_decoded, target_names=label_encoder.classes_)
print(report)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0                drug
1               other
2                drug
3             disease
4             disease
            ...      
715             other
716    treatment plan
717    treatment plan
718    treatment plan
719    treatment plan
Name: Manual_Intent, Length: 720, dtype: object


Step,Training Loss,Validation Loss,Accuracy
90,No log,1.357756,0.315353
180,No log,1.234995,0.423237
270,No log,1.294584,0.394191
360,No log,1.198205,0.481328
450,No log,1.142437,0.497925
540,1.320300,1.097869,0.531120
630,1.320300,1.081129,0.535270
720,1.320300,1.066980,0.580913


{'eval_loss': 1.0669803619384766, 'eval_accuracy': 0.5809128630705395, 'eval_runtime': 22.4658, 'eval_samples_per_second': 10.727, 'eval_steps_per_second': 1.38, 'epoch': 8.0}
                precision    recall  f1-score   support

       disease       0.50      0.70      0.59        76
          drug       0.53      0.77      0.63        53
         other       0.78      0.63      0.70        73
treatment plan       0.00      0.00      0.00        39

      accuracy                           0.58       241
     macro avg       0.45      0.53      0.48       241
  weighted avg       0.51      0.58      0.53       241



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
